### Reduce Size of CPI Database just for our needs
This means we're only gonna utilize rows from the following countries (in the IMF CPI Database):
* $ --> United States
* nz$ --> New Zealand
* a(u)$ --> Australia
* hk$ --> China, P.R.: Hong Kong
* nt$ --> Taiwan is not in the Database
* mx$ --> Mexico
* £ --> United Kingdom
* € --> Euro Area
* ¥ --> Japan
* cn¥ --> China, P.R.: Mainland
* ₽ --> Russian Federation
* SWK --> Sweden
* ₹ --> India
* ₺ --> Türkiye, Rep of
* rm --> Malaysia

In [1]:
import pandas as pd
from pathlib import Path
import numpy as np

INCLUDED_COUNTRIES: list[str] = [
    "United States",
    "New Zealand",
    "Australia",
    "China, P.R.: Hong Kong",
# "Taiwan is not in the Database",
    "Mexico",
    "United Kingdom",
    "Euro Area",
    "Japan",
    "China, P.R.: Mainland",
    "Russian Federation",
    "Sweden",
    "India",
    "Türkiye, Rep of",
    "Malaysia",
]

In [2]:
# Path to data files (might change)
parent_path = Path(".", "src", "movies", "filter", "CPI_timeSeries").absolute()

df = pd.read_csv(Path(parent_path, "CPI_01-13-2023_17-26-32-20_timeSeries.csv"), low_memory=False)

In [3]:
# Get relevant countries only
_df = [df.loc[df["Country Name"] == INCLUDED_COUNTRIES[0]]]
for country in INCLUDED_COUNTRIES[1:]:
    _df.append(df.loc[df["Country Name"] == country])

df = pd.concat(_df)
del _df

In [4]:
# Get PCI Information only
_df = df.loc[df["Indicator Name"].str.contains("Consumer Price Index") & ~df["Indicator Name"].str.contains("Percentage")]
_df = _df.loc[_df["Attribute"] == "Value"]

#for row in df.loc[df["Country Name"] == "United States"].iterrows():
#    print(row)
df = _df
del _df

In [5]:
# get columns with PCI information only (and name of Country) 
df = df.filter(regex="(Country Name|(\d{4}(M\d+)?))$")

# merge multiple rows with PCI into one (one holds only months, one only years)
_df = []
for country in INCLUDED_COUNTRIES: 
    search_frame = df.loc[df["Country Name"] == country]
    base_frame = pd.DataFrame(search_frame[0:1]).reset_index(drop=True)
    _df.append(base_frame.combine_first(pd.DataFrame(search_frame[1:2]).reset_index(drop=True)))

df = pd.concat(_df)
del _df

In [6]:
df.filter(regex="(Country Name|(\d{4}))$")

,Country Name,1950,1951,1952,1953,1954,1955,1956,1957,1958,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,United States,NaN,NaN,NaN,NaN,NaN,12.2789840201233,12.46624521433,12.8828058300142,13.2343982762798,...,106.833848874866,108.566932118964,108.695721960693,110.06700893427,112.411557302308,115.157303224791,117.244195476228,118.690501577198,124.266413825838,NaN
0,New Zealand,2.92168775,3.24373125,3.495199,3.65347975,3.82203875,3.91796675,4.054554,4.14273925,4.326948,...,96.34992,97.5326225,97.818105,98.450245,100.27235,101.875,103.525,105.3,109.45,NaN
0,Australia,4.375,5.225,6.125,6.4,6.5,6.6,6.975,7.175,7.225,...,103.5,106.075,107.675,109.05,111.175,113.3,115.125,116.1,119.425,NaN
0,"China, P.R.: Hong Kong",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,84.5833333333333,88.325,90.9666666666667,93.1583333333333,94.55,96.825,99.6166666666667,99.8666666666667,101.433333333333,NaN
0,Mexico,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,82.0362433333333,85.332965,87.6545683333333,90.1279241666667,95.5729641666667,100.255418333333,103.900666666667,107.43,113.541916666667,NaN
0,United Kingdom,NaN,NaN,NaN,NaN,NaN,4.96350908333333,5.2107145,5.40305783333333,5.564762,...,98.2,99.625,99.9916666666667,101,103.583333333333,105.958333333333,107.8,108.866666666667,111.608333333333,NaN
0,Euro Area,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,99.3775,99.8066666666667,99.9991666666667,100.234166666667,101.775833333333,103.561666666667,104.8,105.064166666667,107.783333333333,116.820833333333
0,Japan,NaN,NaN,NaN,NaN,NaN,16.6800100621762,16.7459934818653,17.2648515647668,17.1823755647668,...,94.8333333333333,97.45,98.225,98.1,98.575,99.55,100.016666666667,99.9916666666667,99.7583333333333,NaN
0,"China, P.R.: Mainland",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,86.7127083333333,88.3790158333333,89.6490433333333,91.4420258333333,92.8988216666667,94.8262775,97.5755133333333,99.9362766666667,100.916666666667,NaN
0,Russian Federation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,121.638958333333,131.155275,151.529466666667,162.20085,168.175241666667,173.015825,180.750266666667,186.862625,199.372066666667,NaN


### Data Modification
As one can clearly see, the database is missing a lot of entries.
It may not be entirely accurate but for our purposes it might be useful to linearily fill in missing entries 

In [7]:
# fill in years
MAX_YEAR = 2022
MIN_YEAR = 1950

lin_step = np.float64(0.0)
_df = []
for country in INCLUDED_COUNTRIES: 
    co_df = df.loc[df["Country Name"] == country].copy()

    # years
    ## backwards
    for year in range(MAX_YEAR, MIN_YEAR, -1):
        next_year = np.float64(co_df[str(year)][0])
        last_year = np.float64(co_df[str(year-1)][0])
        if np.isnan(next_year):
            continue
        elif np.isnan(last_year):
            last_year = np.max([next_year + lin_step, 0.1]) # cap at CPI of 0.1
            co_df[str(year-1)][0] = last_year
        # finally
        lin_step = last_year - next_year
    ## for 2022 if not existent
    year_22 = np.float64(co_df["2022"][0])
    if np.isnan(year_22):
        co_df["2022"][0] = np.float64(co_df["2021"][0]) + (np.float64(co_df["2021"][0]) - np.float64(co_df["2020"][0]))

    # months
    for year in range(MIN_YEAR, MAX_YEAR+1):
        # continue lin_step for 2022 prediction 
        next_year = np.float64(co_df[str(year+1)][0]) if year != 2022 else np.float64(co_df[str(year)][0]) + 12*lin_step
        cur_year = np.float64(co_df[str(year)][0])
        lin_step = (next_year-cur_year) / 12.0
        for month in range(1, 13):
            mo_val = np.float64(co_df[f"{year}M{month}"][0])
            if not np.isnan(mo_val):
                continue
            co_df[f"{year}M{month}"][0] = cur_year + (month-1)*lin_step
    


    _df.append(co_df.copy())


df = pd.concat(_df).reset_index()
del _df
    

In [8]:
df

,index,Country Name,1950,1950M1,1950M2,1950M3,1950M4,1950M5,1950M6,1950M7,...,2022M3,2022M4,2022M5,2022M6,2022M7,2022M8,2022M9,2022M10,2022M11,2022M12
0,0,United States,11.342678,11.342678,11.358283,11.373888,11.389493,11.405098,11.420704,11.436309,...,131.849001744969,132.585052887912,134.046607400409,135.887881754874,135.871830795371,135.823677916861,136.115805379823,136.667958386741,136.529920135011,134.953579
1,0,New Zealand,2.92168775,2.921688,2.948525,2.975362,3.002199,3.029036,3.055873,3.082709,...,114.291667,114.6375,114.983333,115.329167,115.675,116.020833,116.366667,116.7125,117.058333,117.404167
2,0,Australia,4.375,4.375,4.445833,4.516667,4.5875,4.658333,4.729167,4.8,...,123.304167,123.58125,123.858333,124.135417,124.4125,124.689583,124.966667,125.24375,125.520833,125.797917
3,0,"China, P.R.: Hong Kong",0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,...,103.3,103.1,102.9,103.2,103.3,103.4,103.5,103.8,104,104.436111
4,0,Mexico,1.687709,1.687709,1.680358,1.673008,1.665657,1.658307,1.650956,1.643606,...,120.159,120.809,121.022,122.044,122.948,123.803,124.571,125.276,125.997,125.256424
5,0,United Kingdom,3.727482,3.727482,3.748082,3.768683,3.789283,3.809884,3.830484,3.851085,...,116.5,119,119.7,120.5,121.2,121.8,122.3,124.3,124.8,116.863194
6,0,Euro Area,19.953333,19.953333,20.047153,20.140972,20.234792,20.328611,20.422431,20.51625,...,114.46,115.11,116.07,117.01,117.14,117.85,119.26,121.03,120.95,120.53
7,0,Japan,16.350093,16.350093,16.355592,16.36109,16.366589,16.372087,16.377586,16.383085,...,101.1,101.5,101.8,101.8,102.3,102.7,103.1,103.7,103.9,99.311111
8,0,"China, P.R.: Mainland",0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,...,102.5,102.9,102.7,102.7,103.2,103.1,103.4,103.5,103.3,102.795748
9,0,Russian Federation,2.002573,2.002573,1.998706,1.994839,1.990972,1.987105,1.983238,1.979371,...,227.82,215.008869,216.051322,217.093776,218.136229,219.178683,220.221136,221.26359,222.306043,223.348497


### Export simplified Database

In [9]:
df.to_csv(Path(parent_path, "CPI_simple.csv"))